<h1> Data Extraction </h1>
We extract all the data from all the replay files and store it in a sensible way. We use a modified version of py-slippi that handles the lack of metadata in the ranked dataset.

In [ ]:
import os as os
from pathlib import Path
import numpy as np
import pandas as pd
# import polars as pl
import tqdm
import slippi as slp
from joblib import Parallel, delayed
from multiprocessing import Manager
# import pyarrow as pa
# import pyarrow.parquet as pq
import gzip
import pickle
import feather
import uuid

<h2> Define paths and list files </h2>
We define paths to our datasets and list all files in the directory.

In [ ]:
ranked_path = 'D:\\ranked\\ranked-anonymized-1-116248\\ranked-anonymized'
public_path = 'C:\\Users\\jaspa\\Grant ML\\Slippi_Public_Dataset_v3'
mango_path = 'D:\\Mango'

ranked_files = [os.path.join(ranked_path,file) for file in os.listdir(ranked_path) if file.endswith('.slp')] 
public_files = [os.path.join(public_path,file) for file in os.listdir(public_path) if file.endswith('.slp')] 
mango_files = []
# Walk through the directory tree
for mango_path, dirnames, filenames in os.walk(mango_path):
    for filename in filenames:
        # Check if the file ends with .slp
        if filename.endswith('.slp'):
            # Construct the full file path and add it to the list
            file_path =os.path.join(mango_path,filename)
            mango_files.append(file_path)

print('number of ranked games: ', len(ranked_files))
print('number of public games: ', len(public_files))
print('number of mango games: ', len(mango_files))
# Function to create necessary directories
def create_directories(path):
    if not os.path.exists(path):
        os.makedirs(path)

ranked_frame_data_path = 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data'
public_frame_data_path = 'C:\\Users\\jaspa\\Grant ML\\public_frame_data'
mango_frame_data_path = 'C:\\Users\\jaspa\\Grant ML\\mango_frame_data'

create_directories(ranked_frame_data_path)
create_directories(public_frame_data_path)
create_directories(mango_frame_data_path)

# print(ranked_files[:2])
# print(public_files[:2])
# print(mango_files[:2])



<h2> Extract Simple Data </h2>
Extract the data that will be stored in the data frame. Need to handle the cases where the data is none or not there. Esspecially metadata.

In [ ]:
#get the overall metadata from the start of the game
def get_event_start_data(game,occupied_ports):
    ################
    # Start Metadata
    ################
    start_metadata_dict = {
        'is_teams': game.start.is_teams, #bool # We only take two player games
    }
    
    for i, port in enumerate(occupied_ports):
        start_metadata_dict.update({
            f'player_{i+1}_port': port, #int
            f'player_{i+1}_character_name': game.start.players[port].character.name, #string
            # # f'player_{i+1}_character_value': game.start.players[port_1].character.value, #int
            f'player_{i+1}_type_name': game.start.players[port].type.name, #string
            # # f'player_{i+1}_type_value': game.start.players[port].type.value, #int
            f'player_{i+1}_stocks': game.start.players[port].stocks, #int
            f'player_{i+1}_costume': game.start.players[port].costume, #int
            f'player_{i+1}_team_value': game.start.players[port].team.value if game.start.players[port].team else None, #int
            f'player_{i+1}_ucf_shield_drop_name': game.start.players[port].ucf.shield_drop.name, #string
            # f'player_{i+1}_ucf_shield_drop_value': game.start.players[port_1].ucf.shield_drop.value, #int
            f'player_{i+1}_tag': game.start.players[port].tag, #string
            f'player_{i+1}_display_name': game.start.players[port].display_name #string
        })
                # frame.ports[port].leader.post.state_age if frame.ports[port].leader.post.state_age else None,
    start_metadata_dict.update({
        # 'players': game.start.players, #tuple that contains players by port.
        'random_seed': game.start.random_seed, #int #
        'slippi': str(game.start.slippi.version), #slippi TUPLE NEEDS PROCESSING
        'stage_name': game.start.stage.name, #string
        # 'stage_value': game.start.stage.value, #int
        'is_pal': game.start.is_pal, #bool
        'is_frozen_ps': game.start.is_frozen_ps, #bool
    })
    return start_metadata_dict

# Extract all the data from the slippi.event.end module
def get_event_end_data(game):
    end_data_dict = {
        'end_method_name' : game.end.method, #string
        # 'end_method_value' : game.end.method.value, #int
        'lras_initiator' : game.end.lras_initiator #int
    }
    return end_data_dict

# Function to determine who won the game
# 0 if inconclusive
# indexed placement positions
# -1 if inconclusive
def player_placement(game,occupied_ports):
    
    #code to figure out player placement
    #output a dictionary with the placement of each player in the game
    #Only output the players in the game
    
    #Temp code to test
    player_placements = {}
    for i, port in enumerate(occupied_ports):
        player_placements.update({f'player_{i+1}_placement': i+1}) #int

    return player_placements

# Some games won't have metadata
def get_metadata(game, occupied_ports):
    metadata_dict = {
       'date': game.metadata.date, #datetime
       'duration': game.metadata.duration, #int
       'platform': game.metadata.platform.name #string
        }
    
    for i, port in enumerate(occupied_ports):
        metadata_dict.update({
            # f'player_{i+1}_character': game.metadata.players[port].characters if game.metadata.players[port] else None, #string
            f'player_{i+1}_netplay_code': game.metadata.players[port].netplay.code if game.metadata.players[port] else None, #string
            f'player_{i+1}_netplay_name': game.metadata.players[port].netplay.name if game.metadata.players[port] else None, #string
        })
        
    metadata_dict.update({
        'console_name': game.metadata.console_name
        })
        
    return metadata_dict


<h2> Frame Data Functions </h2>
These are the functions that we will call to extract all the frame data and one-hot encode some of it. The outputs are numpy arrays. I am not removing the first 123 frames.

In [ ]:
# # Function to extract frames.pre data
# def get_frames_pre_data(frames, port):
#     # self.state = state #: :py:class:`slippi.id.ActionState` | int: Character's action state
#     # self.position = position #: :py:class:`Position`: Character's position
#     # self.direction = direction #: :py:class:`Direction`: Direction the character is facing
#     # self.joystick = joystick #: :py:class:`Position`: Processed analog joystick position
#     # self.cstick = cstick #: :py:class:`Position`: Processed analog c-stick position
#     # self.triggers = triggers #: :py:class:`Triggers`: Trigger state
#     # self.buttons = buttons #: :py:class:`Buttons`: Button state
#     # self.random_seed = random_seed #: int: Random seed at this point
#     # self.raw_analog_x = raw_analog_x #: int | None: `added(1.2.0)` Raw x analog controller input (for UCF)
#     # self.damage = damage #: float | None: `added(1.4.0)` Current damage percent
    
#     pre_data = np.empty((4,len(frames)),dtype = np.float)
#     # self.state = state #: :py:class:`slippi.id.ActionState` | int: Character's action state
#     # self.position = position #: :py:class:`Position`: Character's position
#     # self.direction = direction #: :py:class:`Direction`: Direction the character is facing
#     # self.damage = damage #: float | None: `added(1.4.0)` Current damage percent
    
#     pre_input_data = np.empty((4,len(frames)),dtype = np.float)
    
#     for i, frame in enumerate(frames):
#         integer_data[0,i] = frame.ports[port].leader.pre.state.value
#         integer_data[1,i] = frame.ports[port].leader.pre.direction.value
#         integer_data[2,i] = frame.ports[port].leader.pre.buttons.logical.value
#         integer_data[3,i] = frame.ports[port].leader.pre.buttons.physical.value
        
#         float_data[0,i] = frame.ports[port].leader.pre.position.x
#         float_data[1,i] = frame.ports[port].leader.pre.position.y
#         float_data[2,i] = frame.ports[port].leader.pre.joystick.x
#         float_data[3,i] = frame.ports[port].leader.pre.joystick.y
#         float_data[4,i] = frame.ports[port].leader.pre.cstick.x
#         float_data[5,i] = frame.ports[port].leader.pre.cstick.y  
#         float_data[6,i] = frame.ports[port].leader.pre.trigger.logical
#         float_data[7,i] = frame.ports[port].leader.pre.trigger.physical.l
#         float_data[8,i] = frame.ports[port].leader.pre.trigger.physical.r
    
#     return integer_data, float_data

# # return np containing floats and np containing int.
# def get_frames_post_data(frames,port):
#     class Post(
#         character, #int
#         state, #slippi.id.ActionState: int 0-382
#         position_x, #float
#         position_y,#float
#         direction, #int
#         damage, #float
#         shield, #float
#         stocks, #int
#         last_attack_landed, #int
#         last_hit_by, #int
#         combo_count, #int
#         state_age=None, #float | none
#         flags=None, #int 16-549755813888 int64
#         hit_stun=None,#float
#         airborne=None,#bool
#         ground=None,#int
#         jumps=None, #int
#         l_cancel=None # success = 1, failure = 2
#         )

        

<h2> Get Frames </h2>
A function to put all the frame data from a single port into a pandas df.

In [ ]:
# Function to extract frame data
def get_frames_df(frames, port):
    # Pre
    # self.state = state #: :py:class:`slippi.id.ActionState` | int: Character's action state
    # self.position = position #: :py:class:`Position`: Character's position
    # self.direction = direction #: :py:class:`Direction`: Direction the character is facing
    # self.joystick = joystick #: :py:class:`Position`: Processed analog joystick position
    # self.cstick = cstick #: :py:class:`Position`: Processed analog c-stick position
    # self.triggers = triggers #: :py:class:`Triggers`: Trigger state
    # self.buttons = buttons #: :py:class:`Buttons`: Button state
    # self.random_seed = random_seed #: int: Random seed at this point
    # self.raw_analog_x = raw_analog_x #: int | None: `added(1.2.0)` Raw x analog controller input (for UCF)
    # self.damage = damage #: float | None: `added(1.4.0)` Current damage percent
    
    # Post
    # character: sid.InGameCharacter #: In-game character (can only change for Zelda/Sheik). Check on first frame to determine if Zelda started as Sheik
    # state: Union[sid.ActionState, int] #: Character's action state
    # position: Position #: Character's position
    # direction: Direction #: Direction the character is facing
    # damage: float #: Current damage percent
    # shield: float #: Current size of shield
    # stocks: int #: Number of stocks remaining
    # last_attack_landed: Union[Attack, int] #: Last attack that this character landed
    # last_hit_by: Optional[int] #: Port of character that last hit this character
    # combo_count: int #: Combo count as defined by the game
    # state_age: Optional[float] #: `added(0.2.0)` Number of frames action state has been active. Can have a fractional component for certain actions
    # flags: Optional[StateFlags] #: `added(2.0.0)` State flags
    # hit_stun: Optional[float] #: `added(2.0.0)` Number of hitstun frames remaining
    # airborne: Optional[bool] #: `added(2.0.0)` True if character is airborne
    # ground: Optional[int] #: `added(2.0.0)` ID of ground character is standing on, if any
    # jumps: Optional[int] #: `added(2.0.0)` Jumps remaining
    # l_cancel: Optional[LCancel] #: `added(2.0.0)` L-cancel status, if any
    column_names = ['frame_index',
                    # 
                    'pre_state', 'pre_position_x','pre_position_y','pre_direction',
                    'pre_joystick_x','pre_joystick_y', 'pre_cstick_x', 'pre_cstick_y',
                    'pre_triggers_logical','pre_triggers_physical_l','pre_triggers_physical_r',
                    'pre_buttons_logical','pre_buttons_physical',
                    'pre_random_seed','pre_raw_analog_x', 'pre_damage',
                    # 
                    'post_character',
                    'post_state','post_position_x','post_position_y','post_direction',
                    'post_damage','post_sheild','post_stocks',
                    'post_last_attack_landed','post_last_hit_by','post_combo_count',
                    'post_state_age','post_flags','post_hit_stun',
                    'post_airbourn', 'post_ground','post_jumps','post_l_cancel'
                    ]
    
    frame_data = []
    
    for frame in frames:
        frame_data.append([
            frame.index, # To remind us that it starts at -123
            # Pre
            frame.ports[port].leader.pre.state.value,
            frame.ports[port].leader.pre.position.x,
            frame.ports[port].leader.pre.position.y,
            frame.ports[port].leader.pre.direction.value,
            #
            frame.ports[port].leader.pre.joystick.x,
            frame.ports[port].leader.pre.joystick.y,
            frame.ports[port].leader.pre.cstick.x,
            frame.ports[port].leader.pre.cstick.y,
            #
            frame.ports[port].leader.pre.triggers.logical,
            frame.ports[port].leader.pre.triggers.physical.l,
            frame.ports[port].leader.pre.triggers.physical.r,
            #
            frame.ports[port].leader.pre.buttons.logical.value,
            frame.ports[port].leader.pre.buttons.physical.value,
            #
            frame.ports[port].leader.pre.random_seed,
            frame.ports[port].leader.pre.raw_analog_x,
            frame.ports[port].leader.pre.damage,
            # Post
            frame.ports[port].leader.post.character.value,
            #
            frame.ports[port].leader.post.state.value,
            frame.ports[port].leader.post.position.x,
            frame.ports[port].leader.post.position.y,
            frame.ports[port].leader.post.direction.value,
            #
            frame.ports[port].leader.post.damage,
            frame.ports[port].leader.post.shield,
            frame.ports[port].leader.post.stocks,
            #
            frame.ports[port].leader.post.last_attack_landed.value if frame.ports[port].leader.post.last_attack_landed else None,
            frame.ports[port].leader.post.last_hit_by if frame.ports[port].leader.post.last_hit_by else None,
            frame.ports[port].leader.post.combo_count,
            #
            frame.ports[port].leader.post.state_age if frame.ports[port].leader.post.state_age else None,
            frame.ports[port].leader.post.flags.value if frame.ports[port].leader.post.flags else None,
            frame.ports[port].leader.post.hit_stun if frame.ports[port].leader.post.hit_stun else None,
            #
            frame.ports[port].leader.post.airborne if frame.ports[port].leader.post.airborne else None,
            frame.ports[port].leader.post.ground if frame.ports[port].leader.post.ground else None,
            frame.ports[port].leader.post.jumps if frame.ports[port].leader.post.jumps else None,
            frame.ports[port].leader.post.l_cancel.value if frame.ports[port].leader.post.l_cancel else None
            ])

    return pd.DataFrame(frame_data, columns=column_names)
    

<h2> Process Fuction </h2>
A function to process a .slp file path. Save the frame data in character subfolder of public, ranked, and mango depending on which dataset the game comes from. We want each frame data file to be saved with a unique name that is relatively short (I think there is a package to generate unique codes and then we would put underscore followed by the port.)  We will save all the data, even if we don't think will use it.

In [ ]:



# Each iteration of this creates a row of the dataframe and appends it to the dataframe.
def process_slp_file(slp_path_double,no_teams_1_players, no_teams_2_players, no_teams_3_players, no_teams_4_players, teams_1_players, teams_2_players, teams_3_players, teams_4_players,all_data_list):#, dataset_path, save_path, no_teams_2_players,no_teams_3_players,no_teams_4_players,teams_3_players,teams_4_players):
    try:
        # file_path = os.path.join(dataset_path, slp_file)
        # slp_file_name = slp_file.removesuffix('.slp')
        
        # game = slp.Game(file_path)
        game = slp.Game(slp_path_double[0])
        # Get the frame data and save it
        frames = game.frames
        # Get occupied ports
        occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
        game_data_dict = {'path': slp_path_double[0]}
        # Get the game's start data
        game_data_dict.update(get_event_start_data(game,occupied_ports))
        # # Get the game's end data
        game_data_dict.update(get_event_end_data(game))
        # # Determine player placements
        game_data_dict.update(player_placement(game,occupied_ports))
        
        # # Get the game's metadata, hadle the case that there is no metadata
        if game.metadata is not None:
            game_data_dict.update(get_metadata(game, occupied_ports))
            
        for i, port in enumerate(occupied_ports):
            frame_df = get_frames_df(frames, port)
            character = game.start.players[port].character.name
            save_path = os.path.join(slp_path_double[1],character)
            create_directories(save_path)
            file_path = os.path.join(save_path,str(uuid.uuid4()) + '.feather')
            frame_df.to_feather(file_path)
            game_data_dict.update({f'player_{i+1}_frame_data_path': file_path})

            #save the data frame as either parquet or feather depending on which is faster to open
            #append the path to the frame data for this port to game_data_dict
        
        if game.start.is_teams:
            if len(occupied_ports) == 1: 
                teams_1_players.append(game_data_dict)
            elif len(occupied_ports) == 2:
                teams_2_players.append(game_data_dict)
            elif len(occupied_ports) == 3:
                teams_3_players.append(game_data_dict)
            elif len(occupied_ports) == 4:
                teams_4_players.append(game_data_dict)
        else:
            if len(occupied_ports) == 1: 
                no_teams_1_players.append(game_data_dict)
            elif len(occupied_ports) == 2:
                no_teams_2_players.append(game_data_dict)
            elif len(occupied_ports) == 3:
                no_teams_3_players.append(game_data_dict)
            elif len(occupied_ports) == 4:
                no_teams_4_players.append(game_data_dict)
        
        all_data_list.append(game_data_dict)
        
        return 
             
    except Exception as e:
        print(f"Error processing {slp_path_double[0]}: {str(e)}")

In [ ]:
num_files = 100

# Create triples
slp_paths_doubles = []


# Helper function to add files to the triples list
def add_to_triples(files, frame_data_path,slp_paths_doubles):
    for file in files:
        slp_paths_doubles.append((file, frame_data_path))

# Add files to triples list
add_to_triples(ranked_files[:num_files], ranked_frame_data_path,slp_paths_doubles)
add_to_triples(public_files[:num_files], public_frame_data_path,slp_paths_doubles)
add_to_triples(mango_files[:num_files], mango_frame_data_path,slp_paths_doubles)

# # Print the first few triples to verify
# for slp_path_double in slp_paths_doubles:  # Print more or less based on your need
#     print(slp_path_double)

manager = Manager()
no_teams_1_players = manager.list()
no_teams_2_players = manager.list()
no_teams_3_players = manager.list()
no_teams_4_players = manager.list()
teams_1_players = manager.list()
teams_2_players = manager.list()
teams_3_players = manager.list()
teams_4_players = manager.list()
all_data_list = manager.list()



# Use joblib to parallelize processing of SLP files
Parallel(n_jobs=-1, verbose=1)(delayed(process_slp_file)(slp_path_double,no_teams_1_players, no_teams_2_players, no_teams_3_players, no_teams_4_players, teams_1_players, teams_2_players, teams_3_players, teams_4_players,all_data_list) for slp_path_double in tqdm.tqdm(slp_paths_doubles))


In [ ]:
print(no_teams_1_players)
print(no_teams_2_players)
print(no_teams_3_players)
print(no_teams_4_players)
print(teams_1_players)
print(teams_2_players)
print(teams_3_players)
print(teams_4_players)
print(all_data_list)

print(len(all_data_list))

In [ ]:
no_teams_1_players_df = pd.DataFrame(list(no_teams_1_players))
no_teams_2_players_df = pd.DataFrame(list(no_teams_2_players))
no_teams_3_players_df = pd.DataFrame(list(no_teams_3_players))
no_teams_4_players_df = pd.DataFrame(list(no_teams_4_players))
teams_1_players_df = pd.DataFrame(list(teams_1_players))
teams_2_players_df = pd.DataFrame(list(teams_2_players))
teams_3_players_df = pd.DataFrame(list(teams_3_players))
teams_4_players_df = pd.DataFrame(list(teams_4_players))
all_game_data_df = pd.DataFrame(list(all_data_list))
print(all_game_data_df.shape)
all_game_data_df

In [ ]:
save_path = 'C:\\Users\\jaspa\\Grant ML\\slp\\data'
file_path = os.path.join(save_path, 'all_game_data_df' + '.feather')
all_game_data_df.to_feather(file_path)
file_path = os.path.join(save_path, 'no_teams_1_players_df' + '.feather')
no_teams_1_players_df.to_feather(file_path)
file_path = os.path.join(save_path, 'no_teams_2_players_df' + '.feather')
no_teams_2_players_df.to_feather(file_path)
file_path = os.path.join(save_path, 'no_teams_3_players_df' + '.feather')
no_teams_3_players_df.to_feather(file_path)
file_path = os.path.join(save_path, 'no_teams_4_players_df' + '.feather')
no_teams_4_players_df.to_feather(file_path)
file_path = os.path.join(save_path, 'teams_1_players_df' + '.feather')
teams_1_players_df.to_feather(file_path)
file_path = os.path.join(save_path, 'teams_2_players_df' + '.feather')
teams_2_players_df.to_feather(file_path)
file_path = os.path.join(save_path, 'teams_3_players_df' + '.feather')
teams_3_players_df.to_feather(file_path)
file_path = os.path.join(save_path, 'teams_4_players_df' + '.feather')
teams_4_players_df.to_feather(file_path)

